### Library Imports 

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# Load environment variables from .env file
import os
from dotenv import load_dotenv
from huggingface_hub import login
import base64
from smolagents import ToolCallingAgent, DuckDuckGoSearchTool, HfApiModel,CodeAgent,tool
import numpy as np
import time
import datetime
from smolagents import (

    FinalAnswerTool,
    HfApiModel,
    Tool,
    tool,
    VisitWebpageTool,
    LiteLLMModel,
)



from opentelemetry.sdk.trace import TracerProvider
from openinference.instrumentation.smolagents import SmolagentsInstrumentor
from opentelemetry.exporter.otlp.proto.http.trace_exporter import (
    OTLPSpanExporter,
)
from opentelemetry.sdk.trace.export import SimpleSpanProcessor

# Load variables from .env file
load_dotenv()

In [ ]:
# Get HF_TOKEN from environment variables
hf_token = os.getenv("HF_TOKEN")

# Login with the token from .env

if hf_token:
    login(token=hf_token)
else:
    print("HF_TOKEN not found in .env file. Using interactive login instead.")
    login()

In [ ]:
inference_model = LiteLLMModel(model_id='gemini/gemini-2.0-flash')


### Selecting Playlist for the Party Using Smolagents

In [ ]:
agent = CodeAgent(tools=[DuckDuckGoSearchTool()], model=inference_model)

agent.run(
    "Search for the best music recommendations for a party at the Wayne's mansion."
)

### Custom Tool for Preparing the Food menu 

In [ ]:
# The flags available in vscode like #NOTE are:

In [ ]:
# NOTE: Precising allowed values in the docstring helps direct agent to occasion argument values which exist and limit hallucinations.

In [ ]:
# Tool to suggest a menu based on the occasion
@tool
def suggest_menu(occasion: str) -> str:
    """
    Suggests a menu based on the occasion.
    Args:
        occasion (str): The type of occasion for the party. Allowed values are:
                        - "casual": Menu for casual party.
                        - "formal": Menu for formal party.
                        - "superhero": Menu for superhero party.
                        - "custom": Custom menu.
    """
    if occasion == "casual":
        return "Pizza, snacks, and drinks."
    elif occasion == "formal":
        return "3-course dinner with wine and dessert."
    elif occasion == "superhero":
        return "Buffet with high-energy and healthy food."
    else:
        return "Custom menu for the butler."


# Alfred, the butler, preparing the menu for the party
agent = CodeAgent(tools=[suggest_menu], model=inference_model)

# Preparing the menu for the party
agent.run("Prepare a casual meal for the party.")

### Using Python Imports Inside the Agent 

In [ ]:
agent = CodeAgent(
    tools=[], model=inference_model, additional_authorized_imports=["datetime"]
)

agent.run(
    """
    Alfred needs to prepare for the party. Here are the tasks:
    1. Prepare the drinks - 30 minutes
    2. Decorate the mansion - 60 minutes
    3. Set up the menu - 45 minutes
    4. Prepare the music and playlist - 45 minutes

    If we start right now, at what time will the party be ready?
    """
)

### End to End Agent

In [ ]:
from huggingface_hub import HfApi
api = HfApi()

print("id, likes, downloads")
for model in api.list_models(
    filter="text-generation-inference", sort="likes7d", limit=5
):
    print(model.id, model.likes, model.downloads)

In [ ]:
from smolagents import (
    CodeAgent,
    DuckDuckGoSearchTool,
    FinalAnswerTool,
    HfApiModel,
    Tool,
    tool,
    VisitWebpageTool,
    LiteLLMModel,
)


@tool
def suggest_menu(occasion: str) -> str:
    """
    Suggests a menu based on the occasion.
    Args:
        occasion: The type of occasion for the party.
    """
    if occasion == "casual":
        return "Pizza, snacks, and drinks."
    elif occasion == "formal":
        return "3-course dinner with wine and dessert."
    elif occasion == "superhero":
        return "Buffet with high-energy and healthy food."
    else:
        return "Custom menu for the butler."


@tool
def catering_service_tool(query: str) -> str:
    """
    This tool returns the highest-rated catering service in Gotham City.

    Args:
        query: A search term for finding catering services.
    """
    # Example list of catering services and their ratings
    services = {
        "Gotham Catering Co.": 4.9,
        "Wayne Manor Catering": 4.8,
        "Gotham City Events": 4.7,
    }

    # Find the highest rated catering service (simulating search query filtering)
    best_service = max(services, key=services.get)

    return best_service


class SuperheroPartyThemeTool(Tool):
    name = "superhero_party_theme_generator"
    description = """
    This tool suggests creative superhero-themed party ideas based on a category.
    It returns a unique party theme idea."""

    inputs = {
        "category": {
            "type": "string",
            "description": "The type of superhero party (e.g., 'classic heroes', 'villain masquerade', 'futuristic Gotham').",
        }
    }

    output_type = "string"

    def forward(self, category: str):
        themes = {
            "classic heroes": "Justice League Gala: Guests come dressed as their favorite DC heroes with themed cocktails like 'The Kryptonite Punch'.",
            "villain masquerade": "Gotham Rogues' Ball: A mysterious masquerade where guests dress as classic Batman villains.",
            "futuristic Gotham": "Neo-Gotham Night: A cyberpunk-style party inspired by Batman Beyond, with neon decorations and futuristic gadgets.",
        }

        return themes.get(
            category.lower(),
            "Themed party idea not found. Try 'classic heroes', 'villain masquerade', or 'futuristic Gotham'.",
        )


# Alfred, the butler, preparing the menu for the party
agent = CodeAgent(
    tools=[
        DuckDuckGoSearchTool(),
        VisitWebpageTool(),
        suggest_menu,
        catering_service_tool,
        SuperheroPartyThemeTool(),
    ],
    model=inference_model,
    max_steps=10,
    verbosity_level=2,
)

agent.run(
    "Give me the best playlist for a party at the Wayne's mansion. The party idea is a 'villain masquerade' theme"
)

### Inspecting Agent Behavior using OpenTelemetry & Langfuse 

In [ ]:
LANGFUSE_PUBLIC_KEY = os.environ["LANGFUSE_PUBLIC_KEY"]
LANGFUSE_SECRET_KEY = os.environ["LANGFUSE_SECRET_KEY"]
LANGFUSE_AUTH = base64.b64encode(
    f"{LANGFUSE_PUBLIC_KEY}:{LANGFUSE_SECRET_KEY}".encode()
).decode()

os.environ["OTEL_EXPORTER_OTLP_ENDPOINT"] = (
    "https://cloud.langfuse.com/api/public/otel"  # EU data region
)
os.environ["OTEL_EXPORTER_OTLP_HEADERS"] = (
    f"Authorization=Basic {LANGFUSE_AUTH}"
)

In [ ]:
trace_provider = TracerProvider()
trace_provider.add_span_processor(SimpleSpanProcessor(OTLPSpanExporter()))

SmolagentsInstrumentor().instrument(tracer_provider=trace_provider)

In [ ]:
agent.run(
    "Give me the best playlist for a party at the Wayne's mansion. The party idea is a 'villain masquerade' theme & Also suggest a menu for the party too"
)

### Tool Calling Agents

In [ ]:
agent = ToolCallingAgent(tools=[DuckDuckGoSearchTool()], model=inference_model)

agent.run(
    "Search for the best music recommendations for a party at the Wayne's mansion."
)